# LogicalCircuit To Tket Circuit Conversion

In [ ]:
from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code

from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.passes import DecomposeClassicalExp
from pytket.circuit.display import render_circuit_jupyter as draw

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Decompose
from LogicalQ.Transpilation.UnBox import UnBoxTask
from LogicalQ.Transpilation.DecomposeIfElseOps import DecomposeIfElseOpsTask
from LogicalQ.Transpilation.FlattenIfElseOps import FlattenIfElseOpsTask

In [ ]:
lqc = LogicalCircuit(1, **steane_code)

# Check encode
lqc.encode(0)

for _ in range(64):
    # Check transversal gates
    lqc.x(0)
    lqc.z(0)
    lqc.y(0)
    
    # # Check complex gates
    lqc.h(0)
    lqc.s(0)
    lqc.sdg(0)
    lqc.t(0)
    lqc.tdg(0)

# Check QEC cycle
lqc.append_qec_cycle()

# Check measurements
lqc.measure_all()

lqc.draw("mpl")

In [ ]:
pm = PassManager([UnBoxTask(), FlattenIfElseOpsTask(lqc.cbit_setter_qreg), Decompose()])
lqc_transpiled = pm.run(lqc)
# lqc_transpiled.draw("mpl")

In [ ]:
circuit = qiskit_to_tk(lqc_transpiled)
_ = DecomposeClassicalExp().apply(circuit)

In [ ]:
backend = QuantinuumBackend(device_name="H2-1E")
circuit_compiled = backend.get_compiled_circuit(circuit, optimisation_level=3)

In [ ]:
circuit_compiled.remove_blank_wires()

In [ ]:
cost = backend.cost(circuit_compiled, n_shots=int(1E3), syntax_checker="H2-1SC")
print(cost)